In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import requests
from pprint import pprint

import pandas as pd
%matplotlib inline

import config


# Fetch the service account key JSON file contents
cred = credentials.Certificate(config.firebase_file_path)
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': config.databaseURL
})


In [2]:
#Imports
import pandas as pd
import numpy as np
from scipy.stats import beta

In [3]:
#Get data from database
listingStatsDB  = "listing_stats.json"
r = requests.get(config.databaseURL + listingStatsDB)
r = r.json()
# style_in_html = ""
# data_in_html = ""
if r:
    data = [r[i] for i in r]
    listing_stats_df = pd.DataFrame.from_dict(data, orient='columns')

listing_stats_df = listing_stats_df.astype({"listingID": int,"revenue":float,"sold":int,"visits":int})

    
listing_stats_df.sample(20)

,listingID,record_date,revenue,sold,visits
2574,680077296,2019-07-25,0.0,0,0
2781,702303569,2019-08-10,0.0,0,0
768,680077296,6/12/19,0.0,0,1
2162,693930997,7/12/19,0.0,0,1
207,701339360,7/2/19,0.0,0,1
2338,706002567,7/18/19,0.0,0,0
1162,680082346,5/25/19,0.0,0,0
426,680079806,6/25/19,0.0,0,1
758,691796412,6/12/19,0.0,0,0
3028,721874971,2019-08-19,0.0,0,0


In [4]:
listing_stats_df.dtypes

listingID        int64
record_date     object
revenue        float64
sold             int64
visits           int64
dtype: object

In [5]:
#Get data from database
listingCreationDB  = "listing_creation_dates.json"
r = requests.get(config.databaseURL + listingCreationDB)
r = r.json()
# style_in_html = ""
# data_in_html = ""
if r:
    data = [r[i] for i in r]
    listing_creation_df = pd.DataFrame.from_dict(data, orient='columns')

listing_creation_df = listing_creation_df.astype({"ListingID": int})

listing_creation_df.head()

,ListingID,OriginallyCreated
0,680077296,2019-03-26
1,680079806,2019-03-26
2,680082346,2019-03-26
3,680084324,2019-03-26
4,680085112,2019-03-27


In [6]:
listing_creation_df=listing_creation_df.drop_duplicates()

In [7]:
listing_creation_df['OriginallyCreated'] = pd.to_datetime(listing_creation_df['OriginallyCreated'])
listing_creation_df.dtypes

ListingID                     int64
OriginallyCreated    datetime64[ns]
dtype: object

In [8]:
#Get data from database
activeListingsDB  = "active_listings.json"
r = requests.get(config.databaseURL + activeListingsDB)
r = r.json()
# style_in_html = ""
# data_in_html = ""
if r:
    data = [r[i] for i in r]
    active_listings_df = pd.DataFrame.from_dict(data, orient='columns')

active_listings_df = active_listings_df.astype({"listingID": int})

active_listings_df.head()

,listingID
0,687898920
1,701341356
2,685451744
3,680071910
4,680079806


In [9]:
from datetime import datetime, date, time, timedelta


listingIDs=active_listings_df['listingID']
from pytz import timezone

now = datetime.now()
now = timezone('US/Central').localize(now)


listingIDsAll=[]
creationDateAll=[]
listingViewsAll=[]
totalSiteViewsAll=[]
avg_views_day=[]
max_views_day=[]
min_views_day=[]
editURLs=[]
daysActive=[]

for listing in listingIDs:
    #sort listing stats by listing
    listingIDsAll.append(listing)
    
    editURL=(f'https://www.etsy.com/your/shops/jetchcreations/tools/listings/view:table,stats:true/{listing}')
    editURLs.append(editURL)
    
    stats_by_listing=listing_stats_df.loc[listing_stats_df['listingID']==listing,:]
    
    #find the sum of views for the listing
    listingViews = stats_by_listing['visits'].sum()
    listingViewsAll.append(listingViews)
    
    #find the creation date for listing
    creation_date_by_listings=listing_creation_df.loc[listing_creation_df['ListingID']==listing,:]
    creation_date_by_listings=creation_date_by_listings.reset_index(drop=True)
    creationDate=creation_date_by_listings['OriginallyCreated'][0]
    creationDate = timezone('US/Central').localize(creationDate)
    creationDate =creationDate + timedelta(hours=5)
    creationDateAll.append(creationDate)
    
    noDaysSinceCreation= now - creationDate

    noDaysSinceCreation = noDaysSinceCreation/ np.timedelta64(1, 'D')

    daysActive.append(noDaysSinceCreation)
    
    #find total site views since the listing was created
    for index, item in listing_stats_df.iterrows():
        try:
            record_date = datetime.strptime(item.record_date, "%m/%d/%y")
            record_date = timezone('US/Central').localize(record_date)

            record_date =record_date + timedelta(hours=5)

            listing_stats_df.loc[index,'record_date_local']=record_date
        except:
            record_date = datetime.strptime(item.record_date, "%Y-%m-%d")
            record_date = timezone('US/Central').localize(record_date)

            record_date =record_date + timedelta(hours=5)

            listing_stats_df.loc[index,'record_date_local']=record_date
    
    total_views_since_creation=listing_stats_df.loc[listing_stats_df['record_date_local']>=creationDate]
    
    #find the sum of views for the listing
    total_views_since_creation = total_views_since_creation.astype({"visits": int})

    totalSiteViews = total_views_since_creation['visits'].sum()
    totalSiteViewsAll.append(totalSiteViews)
    
    averageViewsPerListingPerDay = 0.85
    averageViewsPerDay = 28.5
    
    averageViewsPerListingPerDay/averageViewsPerDay
    
    priorScale = 7 #one week
    priorAlpha = averageViewsPerListingPerDay * priorScale #average views per listing per day (success)
    priorBeta = (averageViewsPerDay * priorScale) - priorAlpha #average total views per day minus average listing views (failures)

    scale = 1 #scale to make graph pretty

    posteriorAlpha = priorAlpha + listingViews
    posteriorBeta = priorBeta + (totalSiteViews - listingViews)
    
    
    ntrials = 10000
    sample = beta.rvs(posteriorAlpha, posteriorBeta, scale=scale, size=ntrials)
    
    avg_views_day1=(np.mean(sample)/scale) * averageViewsPerDay
    avg_views_day.append(avg_views_day1)
    min_views_day1=(np.quantile(sample, .025)/ scale) * averageViewsPerDay
    min_views_day.append(min_views_day1)
    max_views_day1=(np.quantile(sample, .975)/ scale) * averageViewsPerDay
    max_views_day.append(max_views_day1)
    
bayes_df=pd.DataFrame({'listingID':listingIDsAll,
'creationDate':creationDateAll,
'listingViews':listingViewsAll,
'totalSiteViews':totalSiteViewsAll,
'avg_views_day':avg_views_day,
'max_views_day':max_views_day,
'min_views_day':min_views_day,
'edit_url':editURLs,
'days_active':daysActive})

bayes_df["curr_views_day"] = (bayes_df.listingViews / bayes_df.totalSiteViews) * 28.5
bayes_df

,listingID,creationDate,listingViews,totalSiteViews,avg_views_day,max_views_day,min_views_day,edit_url,days_active,curr_views_day
0,687898920,2019-04-26 05:00:00-05:00,499,3110,4.348572,4.704214,4.007038,https://www.etsy.com/your/shops/jetchcreations...,117.38736,4.572830
1,701341356,2019-06-18 05:00:00-05:00,83,1445,1.544461,1.874642,1.246771,https://www.etsy.com/your/shops/jetchcreations...,64.38736,1.637024
2,685451744,2019-04-16 05:00:00-05:00,136,3340,1.142007,1.329981,0.968050,https://www.etsy.com/your/shops/jetchcreations...,127.38736,1.160479
3,680071910,2019-03-26 05:00:00-05:00,180,3739,1.346100,1.541694,1.159259,https://www.etsy.com/your/shops/jetchcreations...,148.38736,1.372025
4,680079806,2019-03-26 05:00:00-05:00,203,3739,1.510094,1.714514,1.319263,https://www.etsy.com/your/shops/jetchcreations...,148.38736,1.547339
5,712179258,2019-08-02 05:00:00-05:00,1,286,0.405191,0.752708,0.166475,https://www.etsy.com/your/shops/jetchcreations...,19.38736,0.099650
6,712150418,2019-08-02 05:00:00-05:00,1,286,0.409558,0.755197,0.162851,https://www.etsy.com/your/shops/jetchcreations...,19.38736,0.099650
7,693930997,2019-03-26 05:00:00-05:00,417,3739,3.058433,3.340595,2.786301,https://www.etsy.com/your/shops/jetchcreations...,148.38736,3.178524
8,701210678,2019-06-18 05:00:00-05:00,15,1445,0.363279,0.534056,0.225404,https://www.etsy.com/your/shops/jetchcreations...,64.38736,0.295848
9,680064082,2019-03-26 05:00:00-05:00,1140,3739,8.292344,8.689078,7.898546,https://www.etsy.com/your/shops/jetchcreations...,148.38736,8.689489


In [10]:
bayes_df.sort_values(by='curr_views_day', ascending=False)

,listingID,creationDate,listingViews,totalSiteViews,avg_views_day,max_views_day,min_views_day,edit_url,days_active,curr_views_day
9,680064082,2019-03-26 05:00:00-05:00,1140,3739,8.292344,8.689078,7.898546,https://www.etsy.com/your/shops/jetchcreations...,148.38736,8.689489
0,687898920,2019-04-26 05:00:00-05:00,499,3110,4.348572,4.704214,4.007038,https://www.etsy.com/your/shops/jetchcreations...,117.38736,4.572830
7,693930997,2019-03-26 05:00:00-05:00,417,3739,3.058433,3.340595,2.786301,https://www.etsy.com/your/shops/jetchcreations...,148.38736,3.178524
10,701339360,2019-06-18 05:00:00-05:00,99,1445,1.816955,2.163122,1.492477,https://www.etsy.com/your/shops/jetchcreations...,64.38736,1.952595
1,701341356,2019-06-18 05:00:00-05:00,83,1445,1.544461,1.874642,1.246771,https://www.etsy.com/your/shops/jetchcreations...,64.38736,1.637024
4,680079806,2019-03-26 05:00:00-05:00,203,3739,1.510094,1.714514,1.319263,https://www.etsy.com/your/shops/jetchcreations...,148.38736,1.547339
3,680071910,2019-03-26 05:00:00-05:00,180,3739,1.346100,1.541694,1.159259,https://www.etsy.com/your/shops/jetchcreations...,148.38736,1.372025
2,685451744,2019-04-16 05:00:00-05:00,136,3340,1.142007,1.329981,0.968050,https://www.etsy.com/your/shops/jetchcreations...,127.38736,1.160479
16,701202082,2019-06-18 05:00:00-05:00,57,1445,1.091231,1.365591,0.844413,https://www.etsy.com/your/shops/jetchcreations...,64.38736,1.124221
25,688465922,2019-04-28 05:00:00-05:00,87,3101,0.802864,0.971184,0.647166,https://www.etsy.com/your/shops/jetchcreations...,115.38736,0.799581


In [11]:
bayes_test=[]

for index, item in bayes_df.iterrows():
    bayesDict={}
    bayesDict['a_listingID']=item[0]
    bayesDict['b_daysActive']=round(item[8],1)
    bayesDict['c_ListingVisits']=round(item[2],4)
    bayesDict['d_TotalSiteVisits']=round(item[3],4)
    bayesDict['e_CurrentVisitsDay']=round(item[9],4)
    bayesDict['f_AvgListingVisits']=round(item[4],4)
    bayesDict['g_MinListingVisits']=round(item[6],4)
    bayesDict['h_MaxListingVisits']=round(item[5],4)
    bayesDict['i_EditURL']=item[7]


    bayes_test.append(bayesDict)
bayes_test

[{'a_listingID': 687898920,
  'b_daysActive': 117.4,
  'c_ListingVisits': 499,
  'd_TotalSiteVisits': 3110,
  'e_CurrentVisitsDay': 4.5728,
  'f_AvgListingVisits': 4.3486,
  'g_MinListingVisits': 4.007,
  'h_MaxListingVisits': 4.7042,
  'i_EditURL': 'https://www.etsy.com/your/shops/jetchcreations/tools/listings/view:table,stats:true/687898920'},
 {'a_listingID': 701341356,
  'b_daysActive': 64.4,
  'c_ListingVisits': 83,
  'd_TotalSiteVisits': 1445,
  'e_CurrentVisitsDay': 1.637,
  'f_AvgListingVisits': 1.5445,
  'g_MinListingVisits': 1.2468,
  'h_MaxListingVisits': 1.8746,
  'i_EditURL': 'https://www.etsy.com/your/shops/jetchcreations/tools/listings/view:table,stats:true/701341356'},
 {'a_listingID': 685451744,
  'b_daysActive': 127.4,
  'c_ListingVisits': 136,
  'd_TotalSiteVisits': 3340,
  'e_CurrentVisitsDay': 1.1605,
  'f_AvgListingVisits': 1.142,
  'g_MinListingVisits': 0.968,
  'h_MaxListingVisits': 1.33,
  'i_EditURL': 'https://www.etsy.com/your/shops/jetchcreations/tools/listi

In [12]:
bayes_df.columns

Index(['listingID', 'creationDate', 'listingViews', 'totalSiteViews',
       'avg_views_day', 'max_views_day', 'min_views_day', 'edit_url',
       'days_active', 'curr_views_day'],
      dtype='object')

# Test for Single Item

In [13]:
test_listingIDs= 721881161
stats_by_listing=listing_stats_df.loc[listing_stats_df['listingID']==test_listingIDs,:]
stats_by_listing.head()

,listingID,record_date,revenue,sold,visits,record_date_local
2265,721881161,7/16/19,0.0,0,0,2019-07-16 05:00:00-05:00
2296,721881161,7/17/19,0.0,0,0,2019-07-17 05:00:00-05:00
2327,721881161,7/18/19,0.0,0,0,2019-07-18 05:00:00-05:00
2358,721881161,7/19/19,0.0,0,0,2019-07-19 05:00:00-05:00
2390,721881161,7/20/19,0.0,0,0,2019-07-20 05:00:00-05:00


In [14]:
listingViews = stats_by_listing['visits'].sum()
listingViews

2

In [15]:
creation_date_by_listings=listing_creation_df.loc[listing_creation_df['ListingID']==test_listingIDs,:]
creation_date_by_listings

,ListingID,OriginallyCreated
25,721881161,2019-07-16


In [16]:
creation_date_by_listings=creation_date_by_listings.reset_index(drop=True)
creationDate=creation_date_by_listings['OriginallyCreated'][0]
creationDate

Timestamp('2019-07-16 00:00:00')

In [17]:
total_views_since_creation=listing_stats_df.loc[listing_stats_df['date']>=creationDate]
total_views_since_creation.head()

KeyError: 'date'

In [ ]:
totalSiteViews = total_views_since_creation['visits'].sum()
totalSiteViews